In [15]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold

In [16]:
# Load datasets
occupancy_df_training = pd.read_csv("occupancy_data/datatraining.txt", header=0)

print(occupancy_df_training.head())

occupancy_df_test1 = pd.read_csv("occupancy_data/datatest.txt", header=0)
occupancy_df_test2 = pd.read_csv("occupancy_data/datatest2.txt", header=0)

spambase_df = pd.read_csv("spambase/spambase.data", header=None)

print(spambase_df.head())

                  date  Temperature  Humidity  Light     CO2  HumidityRatio  \
1  2015-02-04 17:51:00        23.18   27.2720  426.0  721.25       0.004793   
2  2015-02-04 17:51:59        23.15   27.2675  429.5  714.00       0.004783   
3  2015-02-04 17:53:00        23.15   27.2450  426.0  713.50       0.004779   
4  2015-02-04 17:54:00        23.15   27.2000  426.0  708.25       0.004772   
5  2015-02-04 17:55:00        23.10   27.2000  426.0  704.50       0.004757   

   Occupancy  
1          1  
2          1  
3          1  
4          1  
5          1  
     0     1     2    3     4     5     6     7     8     9  ...    48     49  \
0  0.00  0.64  0.64  0.0  0.32  0.00  0.00  0.00  0.00  0.00 ...  0.00  0.000   
1  0.21  0.28  0.50  0.0  0.14  0.28  0.21  0.07  0.00  0.94 ...  0.00  0.132   
2  0.06  0.00  0.71  0.0  1.23  0.19  0.19  0.12  0.64  0.25 ...  0.01  0.143   
3  0.00  0.00  0.00  0.0  0.63  0.00  0.31  0.63  0.31  0.63 ...  0.00  0.137   
4  0.00  0.00  0.00  0.0  0.63

In [17]:
occupancy_y = occupancy_df_training.Occupancy

occupancy_df_training = occupancy_df_training.drop("Occupancy", axis=1)
occupancy_df_training = occupancy_df_training.drop("date", axis=1)

spambase_y = spambase_df[57]
spambase_df = spambase_df.drop(57, axis=1)
spambase_df = spambase_df.iloc[:,0:20]
print(spambase_df.head())

     0     1     2    3     4     5     6     7     8     9     10    11  \
0  0.00  0.64  0.64  0.0  0.32  0.00  0.00  0.00  0.00  0.00  0.00  0.64   
1  0.21  0.28  0.50  0.0  0.14  0.28  0.21  0.07  0.00  0.94  0.21  0.79   
2  0.06  0.00  0.71  0.0  1.23  0.19  0.19  0.12  0.64  0.25  0.38  0.45   
3  0.00  0.00  0.00  0.0  0.63  0.00  0.31  0.63  0.31  0.63  0.31  0.31   
4  0.00  0.00  0.00  0.0  0.63  0.00  0.31  0.63  0.31  0.63  0.31  0.31   

     12    13    14    15    16    17    18    19  
0  0.00  0.00  0.00  0.32  0.00  1.29  1.93  0.00  
1  0.65  0.21  0.14  0.14  0.07  0.28  3.47  0.00  
2  0.12  0.00  1.75  0.06  0.06  1.03  1.36  0.32  
3  0.31  0.00  0.00  0.31  0.00  0.00  3.18  0.00  
4  0.31  0.00  0.00  0.31  0.00  0.00  3.18  0.00  


In [18]:
# Split with KFolds
kf = KFold(10, True, 1)

In [19]:
# Create a model with 2 layers
model = keras.Sequential()
model.add(keras.layers.Dense(16, activation=tf.nn.sigmoid)) # 16 neurons and sigmoid activation function
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid)) # 1 neuron and sigmoid activation function

In [20]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [21]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', f1_m])

In [8]:
for train, test in kf.split(occupancy_df_training):
    
    X_train, X_test, y_train, y_test = occupancy_df_training.iloc[train], occupancy_df_training.iloc[test], occupancy_y.iloc[train], occupancy_y.iloc[test]
    
    history = model.fit(X_train.values,
                        y_train.values,
                        epochs=10)
    
    # Model Accuracy: how often is the classifier correct?
    loss, accuracy, f1 = model.evaluate(X_test, y_test)
    
    print("\naccuracy:   %0.3f, f1_score:   %0.3f\n" % (accuracy, f1))

Epoch 1/10
7328/7328 [==============================] - 1s 147us/step - loss: 0.5905 - acc: 0.7354 - f1_m: 0.0126
Epoch 2/10
7328/7328 [==============================] - 0s 59us/step - loss: 0.5191 - acc: 0.7883 - f1_m: 0.0000e+00
Epoch 3/10
7328/7328 [==============================] - 1s 69us/step - loss: 0.5163 - acc: 0.7883 - f1_m: 0.0000e+00
Epoch 4/10
7328/7328 [==============================] - 0s 62us/step - loss: 0.5163 - acc: 0.7883 - f1_m: 0.0000e+00
Epoch 5/10
7328/7328 [==============================] - 1s 77us/step - loss: 0.5163 - acc: 0.7883 - f1_m: 0.0000e+00
Epoch 6/10
7328/7328 [==============================] - 0s 54us/step - loss: 0.5163 - acc: 0.7883 - f1_m: 0.0000e+00
Epoch 7/10
7328/7328 [==============================] - 0s 60us/step - loss: 0.5163 - acc: 0.7883 - f1_m: 0.0000e+00
Epoch 8/10
7328/7328 [==============================] - 0s 55us/step - loss: 0.5163 - acc: 0.7883 - f1_m: 0.0000e+00
Epoch 9/10
7328/7328 [==============================] - 0s 57us/ste

7329/7329 [==============================] - 0s 52us/step - loss: 0.5198 - acc: 0.7855 - f1_m: 0.0000e+00
Epoch 6/10
7329/7329 [==============================] - 1s 73us/step - loss: 0.5200 - acc: 0.7855 - f1_m: 0.0000e+00
Epoch 7/10
7329/7329 [==============================] - 1s 73us/step - loss: 0.5199 - acc: 0.7855 - f1_m: 0.0000e+00
Epoch 8/10
7329/7329 [==============================] - 1s 84us/step - loss: 0.5198 - acc: 0.7855 - f1_m: 0.0000e+00
Epoch 9/10
7329/7329 [==============================] - 1s 88us/step - loss: 0.5200 - acc: 0.7855 - f1_m: 0.0000e+00
Epoch 10/10
814/814 [==============================] - 0s 26us/step

accuracy:   0.807, f1_score:   0.000

Epoch 1/10
7329/7329 [==============================] - 0s 64us/step - loss: 0.5131 - acc: 0.7907 - f1_m: 0.0000e+00
Epoch 2/10
7329/7329 [==============================] - 0s 58us/step - loss: 0.5133 - acc: 0.7907 - f1_m: 0.0000e+00
Epoch 3/10
7329/7329 [==============================] - 0s 62us/step - loss: 0.5130 -

In [22]:
for train, test in kf.split(spambase_df):
    
    X_train, X_test, y_train, y_test = spambase_df.iloc[train], spambase_df.iloc[test], spambase_y.iloc[train], spambase_y.iloc[test]
    
    history = model.fit(X_train.values,
                        y_train.values,
                        epochs=5)
    
    # Model Accuracy: how often is the classifier correct?
    loss, accuracy, f1 = model.evaluate(X_test, y_test)
    
    print("\naccuracy:   %0.3f, f1_score:   %0.3f\n" % (accuracy, f1))

Epoch 1/5
4140/4140 [==============================] - 1s 189us/step - loss: 0.6021 - acc: 0.6819 - f1_m: 0.2936
Epoch 2/5
4140/4140 [==============================] - 0s 95us/step - loss: 0.5342 - acc: 0.7978 - f1_m: 0.6771
Epoch 3/5
4140/4140 [==============================] - 0s 60us/step - loss: 0.4886 - acc: 0.8155 - f1_m: 0.7250
Epoch 4/5
4140/4140 [==============================] - 0s 55us/step - loss: 0.4560 - acc: 0.8271 - f1_m: 0.7479
Epoch 5/5
461/461 [==============================] - 0s 278us/step

accuracy:   0.844, f1_score:   0.345

Epoch 1/5
4141/4141 [==============================] - 0s 56us/step - loss: 0.4202 - acc: 0.8370 - f1_m: 0.7683
Epoch 2/5
4141/4141 [==============================] - 0s 67us/step - loss: 0.4077 - acc: 0.8406 - f1_m: 0.7702
Epoch 3/5
4141/4141 [==============================] - 0s 59us/step - loss: 0.3986 - acc: 0.8428 - f1_m: 0.7801
Epoch 4/5
4141/4141 [==============================] - 0s 57us/step - loss: 0.3919 - acc: 0.8503 - f1_m: 0.78

In [23]:
# Create a model with 1 layer
model = keras.Sequential()
model.add(keras.layers.Dense(16, activation=tf.nn.sigmoid)) # 16 neurons and sigmoid activation function

In [24]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', f1_m])

In [12]:
for train, test in kf.split(occupancy_df_training):
    
    X_train, X_test, y_train, y_test = occupancy_df_training.iloc[train], occupancy_df_training.iloc[test], occupancy_y.iloc[train], occupancy_y.iloc[test]
    
    history = model.fit(X_train.values,
                        y_train.values,
                        epochs=10)
    
    # Model Accuracy: how often is the classifier correct?
    loss, accuracy, f1 = model.evaluate(X_test, y_test)
    
    print("\naccuracy:   %0.3f, f1_score:   %0.3f\n" % (accuracy, f1))

Epoch 1/10
7328/7328 [==============================] - 1s 98us/step - loss: 6.4794 - acc: 0.5876 - f1_m: 0.4945
Epoch 2/10
7328/7328 [==============================] - 0s 48us/step - loss: 6.2935 - acc: 0.5999 - f1_m: 0.5234
Epoch 3/10
7328/7328 [==============================] - 1s 73us/step - loss: 6.2714 - acc: 0.6019 - f1_m: 0.5269
Epoch 4/10
7328/7328 [==============================] - 0s 51us/step - loss: 6.2680 - acc: 0.6023 - f1_m: 0.5270
Epoch 5/10
7328/7328 [==============================] - 0s 50us/step - loss: 6.2648 - acc: 0.6029 - f1_m: 0.5262
Epoch 6/10
7328/7328 [==============================] - 0s 49us/step - loss: 6.2613 - acc: 0.6031 - f1_m: 0.5285
Epoch 7/10
7328/7328 [==============================] - 0s 47us/step - loss: 6.2550 - acc: 0.6037 - f1_m: 0.5285
Epoch 8/10
7328/7328 [==============================] - 0s 47us/step - loss: 6.2168 - acc: 0.6051 - f1_m: 0.5281
Epoch 9/10
7328/7328 [==============================] - 0s 47us/step - loss: 5.6972 - acc: 0.640

7329/7329 [==============================] - 0s 61us/step - loss: 4.8758 - acc: 0.6934 - f1_m: 0.6556
Epoch 9/10
7329/7329 [==============================] - 0s 50us/step - loss: 4.8760 - acc: 0.6932 - f1_m: 0.6562
Epoch 10/10
814/814 [==============================] - 0s 31us/step

accuracy:   0.695, f1_score:   0.404

Epoch 1/10
7329/7329 [==============================] - 0s 51us/step - loss: 4.8563 - acc: 0.6943 - f1_m: 0.6425
Epoch 2/10
7329/7329 [==============================] - 0s 51us/step - loss: 4.8554 - acc: 0.6944 - f1_m: 0.6443
Epoch 3/10
7329/7329 [==============================] - 0s 49us/step - loss: 4.8553 - acc: 0.6942 - f1_m: 0.6426
Epoch 4/10
7329/7329 [==============================] - 0s 59us/step - loss: 4.8544 - acc: 0.6943 - f1_m: 0.6424
Epoch 5/10
7329/7329 [==============================] - 0s 55us/step - loss: 4.8555 - acc: 0.6944 - f1_m: 0.6407
Epoch 6/10
7329/7329 [==============================] - 0s 49us/step - loss: 4.8552 - acc: 0.6944 - f1_m: 0.6429


In [25]:
for train, test in kf.split(spambase_df):
    
    X_train, X_test, y_train, y_test = spambase_df.iloc[train], spambase_df.iloc[test], spambase_y.iloc[train], spambase_y.iloc[test]
    
    history = model.fit(X_train.values,
                        y_train.values,
                        epochs=10)
    
    # Model Accuracy: how often is the classifier correct?
    loss, accuracy, f1 = model.evaluate(X_test, y_test)
    
    print("\naccuracy:   %0.3f, f1_score:   %0.3f\n" % (accuracy, f1))

Epoch 1/10
4140/4140 [==============================] - 1s 188us/step - loss: 0.7302 - acc: 0.5807 - f1_m: 0.8884
Epoch 2/10
4140/4140 [==============================] - 0s 62us/step - loss: 0.6700 - acc: 0.6625 - f1_m: 1.0598
Epoch 3/10
4140/4140 [==============================] - 0s 61us/step - loss: 0.6282 - acc: 0.7266 - f1_m: 1.2152
Epoch 4/10
4140/4140 [==============================] - ETA: 0s - loss: 0.5989 - acc: 0.7643 - f1_m: 1.32 - 0s 61us/step - loss: 0.5961 - acc: 0.7672 - f1_m: 1.3226
Epoch 5/10
4140/4140 [==============================] - 0s 87us/step - loss: 0.5704 - acc: 0.7920 - f1_m: 1.4066
Epoch 6/10
4140/4140 [==============================] - 0s 80us/step - loss: 0.5493 - acc: 0.8044 - f1_m: 1.4562
Epoch 7/10
4140/4140 [==============================] - 0s 56us/step - loss: 0.5317 - acc: 0.8105 - f1_m: 1.4931
Epoch 8/10
4140/4140 [==============================] - 0s 54us/step - loss: 0.5171 - acc: 0.8166 - f1_m: 1.5109
Epoch 9/10
4140/4140 [=====================

4141/4141 [==============================] - 0s 53us/step - loss: 0.3987 - acc: 0.8351 - f1_m: 1.5961
Epoch 8/10
4141/4141 [==============================] - 0s 48us/step - loss: 0.3985 - acc: 0.8368 - f1_m: 1.6001
Epoch 9/10
4141/4141 [==============================] - 0s 51us/step - loss: 0.3983 - acc: 0.8357 - f1_m: 1.6016 0s - loss: 0.4111 - acc: 0.8305 - f1_m: 
Epoch 10/10
460/460 [==============================] - 0s 34us/step

accuracy:   0.828, f1_score:   0.730

Epoch 1/10
4141/4141 [==============================] - 0s 49us/step - loss: 0.3988 - acc: 0.8382 - f1_m: 1.5984
Epoch 2/10
4141/4141 [==============================] - 0s 53us/step - loss: 0.3986 - acc: 0.8372 - f1_m: 1.5826
Epoch 3/10
4141/4141 [==============================] - 0s 50us/step - loss: 0.3985 - acc: 0.8354 - f1_m: 1.5959
Epoch 4/10
4141/4141 [==============================] - 0s 49us/step - loss: 0.3983 - acc: 0.8350 - f1_m: 1.5972
Epoch 5/10
4141/4141 [==============================] - 0s 49us/step - l